Alunos

Sandoval da Silva Almeida Junior

Tayco Murilo Santos Rodrigues

**Equação de Bellman**:

$$Q^{(i+1)}[s,a] = (1-\alpha)\,Q^{(i)}[s,a] + \alpha \left( r + \gamma \,Q^{(i)}\left[s,\text{argmax}_{a} \,Q^{(i)}[s,a]\right] \right).$$

In [1]:
import gym
import random
import numpy as np

In [ ]:
#Instalar o pacote gym e a extensâo para o atari

env = gym.make("Robotank-ram-v0", render_mode ='human') #Criar uma instância com o gym

In [3]:
class QLearn: 
    
    def __init__(self, actions, epsilon, alpha, gamma):
        
        # Dicionário para receber os valores de q
        self.q = {}
        
        # Constante de Exploração // Determina se a proxima ação é aleatória ou coordenada
        self.epsilon = epsilon  
        
        # Constante de Desconto   // taxa de aprendizado
        self.alpha = alpha  
        
        # Taxa de Desconto        // Pesos atribuídos aos estados
        self.gamma = gamma  
        
        # Ações
        self.actions = actions

    # Método para obter o valor de q, com base em cada ação e estado
    def getQ(self, estado, acao):
        return self.q.get((estado, acao), 0.0)  #get(getQ(), 0.0)

    # Método para o aprendizado de q, onde será implementada a Equação de Bellman
    def aprendeQ(self, estado, acao, recompensa, valor):
        '''
        Algoritmo Q-learning:
            Q(s, a) += alpha * (reward(s,a) + max(Q(s') - Q(s,a))
        '''
        
        # Obtém o valor anterior de q
        antigo_q = self.q.get((estado, acao), None)
        
        # Se o valor estiver vazio, coletamos a recompensa, Se não, geramos um novo valor de q
        if antigo_q is None:
            self.q[(estado, acao)] = recompensa
        else:
            self.q[(estado, acao)] = antigo_q + self.alpha * (valor - antigo_q)
            # Q(s, a)              = Q(s, a)   + alpha     * (reward(s,a) + max(Q(s') - Q(s,a))

    def escolheAcao(self, estado, return_q = False):
        
        # self.actions foi iniciado no construtor da classe
        q = [self.getQ(estado, a) for a in self.actions] # Fixa um estado e roda todas as ações nesse estado
        

        maxQ = max(q) # Grava o maior valor de q no estado enviado para a função

        #Agora introduzimos aleatoriedade às decisões do agente
        if random.random() < self.epsilon:
            minQ = min(q)
            mag = max(abs(minQ), abs(maxQ))
            
            # Adiciona valores aleatórios a todas as ações, recalculando maxQ
            q = [q[i] + random.random() * mag - .5 * mag for i in range(len(self.actions))]
            maxQ = max(q)

        # Conta os valores máximos
        count = q.count(maxQ)
        
        # Caso haja vários valores máximos de ação de estado selecionamos um aleatório entre eles
        if count > 1:
            best = [i for i in range(len(self.actions)) if q[i] == maxQ]
            i = random.choice(best) # Escolhe aleatoriamente a posição da ação na lista que atende a q[i] == maxQ
        else:
            i = q.index(maxQ)
        
        # E alimentamos a variável de ação
        action = self.actions[i] # Melhor ação pra aquele estado
        
        # Por fim, retornamos q e a ação ou apenas a ação
        if return_q:  
            return action, q
        return action

    # Este é o método de aprendizado para o treinamento do agente
    def aprendizado(self, state1, action1, reward, state2):
        maxqnew = max([self.getQ(state2, a) for a in self.actions]) # novo_estado = state2
        self.aprendeQ(state1, action1, reward, reward + self.gamma * maxqnew)
        #def aprendeQ(self, estado, acao, recompensa, valor):

In [4]:
# Função para criar o estado a partir do ambiente do Gym
def cria_estado(atributos):
    return int("".join(map(lambda atributo: str(int(atributo)), atributos)))

In [5]:
# Função para o aprendizado de um episódio
def aprende_um_episodio(Q, episodio):
    
    # Variável de controle do loop de aprendizado
    done = False
    
    # Inicializa variáveis
    G, reward = 0, 0
    
    # Reset do ambiente
    state = env.reset()
    

    while not done:
        
        
        # Escolhe uma ação no estado atual
        action = Q.escolheAcao(cria_estado(state))
        
        # Cada ação gera resultados e coletamos todos eles: estado, recompensa, se chegamos ao final e info
        state2, reward, done, info = env.step(action)
        
        # Usamos os resultados anteriores para o agente aprender se aquela ação foi boa ou não
        Q.aprendizado(cria_estado(state), action, reward, cria_estado(state))
        
        # Vamos acumulando as recompensas
        G += reward
        
        # Para para o próximo estado
        state = state2

    # Andamento do aprendizado
    print('Episódio {} Recompensa Total: {}'.format(episodio, G))

In [6]:
# Cria o objeto Q (instância da Classe) inicializando os parâmetros para as primeiras escolhas do agente

# Hiperparâmetros iniciais
actions = list(range(0, 18))
epsilon = 0.4
alpha = 0.618
gamma = 0.9

# Instância da classe
Q = QLearn(actions, epsilon, alpha, gamma)

In [ ]:
# Primeira rodada de treinamento com 21 episódios
# Cada recompensa faz com o que o agente aprenda a ação que gera o resultado, ou seja, vencer o jogo
for episodio in range(1, 21):
    aprende_um_episodio(Q, episodio)
   

In [ ]:
# Alterar os parâmetros
Q.epsilon = 0.3
Q.alpha = 0.518
Q.gamma = 0.8

# Rodada de treinamento
for episodio in range(1, 21):
    aprende_um_episodio(Q, episodio)